In [38]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup as BS
from bs4 import Comment
import time
from tqdm import tqdm
import datetime as dt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
import numpy as np

# All Game Data:

In [39]:
full_stats = pickle.load(open('dataframe.pkl', 'rb'))

In [40]:
pd.set_option('display.max_columns', 100)

In [41]:
full_stats=full_stats.drop(columns = ['consec_away'])

In [42]:
numerics = []
for col in full_stats.columns:
    if full_stats[col].dtype == 'float64':
        numerics.append(col)

In [43]:
full_stats_model = full_stats.dropna()

In [44]:
full_stats_model['away_losses'] = pd.DataFrame(full_stats_model.groupby([pd.Grouper(key = 'date', freq = '1Y'), 'away_abbr'])['Home_win'].cumsum())
full_stats_model['home_wins'] = pd.DataFrame(full_stats_model.groupby([pd.Grouper(key = 'date', freq = '1Y'), 'home_abbr'])['Home_win'].cumsum())
full_stats_model['num_games'] = 1
full_stats_model['total_home_games'] = pd.DataFrame(full_stats_model.groupby([pd.Grouper(key = 'date', freq = '1Y'), 'home_abbr'])['num_games'].cumsum())
full_stats_model['total_away_games'] = pd.DataFrame(full_stats_model.groupby([pd.Grouper(key = 'date', freq = '1Y'), 'away_abbr'])['num_games'].cumsum())
full_stats_model['away_win_pct'] = (1 - (full_stats_model['away_losses']/ full_stats_model['total_away_games']))
full_stats_model['home_win_pct'] = (full_stats_model['home_wins']/full_stats_model['total_home_games'])
full_stats_model['win_pct_diff'] = full_stats_model['home_win_pct'] - full_stats_model['away_win_pct']


<ipython-input-44-5ec66f9e0b07>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_stats_model['away_losses'] = pd.DataFrame(full_stats_model.groupby([pd.Grouper(key = 'date', freq = '1Y'), 'away_abbr'])['Home_win'].cumsum())
<ipython-input-44-5ec66f9e0b07>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_stats_model['home_wins'] = pd.DataFrame(full_stats_model.groupby([pd.Grouper(key = 'date', freq = '1Y'), 'home_abbr'])['Home_win'].cumsum())
<ipython-input-44-5ec66f9e0b07>:3: SettingWithCopyWarnin

In [45]:
numerics = numerics + ['win_pct_diff']

In [46]:
full_hist_class = HistGradientBoostingClassifier()

In [47]:
full_hist_class.fit(full_stats_model[numerics], full_stats_model['Home_win'])

HistGradientBoostingClassifier()

# Scraping todays games

In [48]:
def get_game(soup, game_id):
    game = {'game_id':game_id}
    score_table = soup.find('div', {'class':'scorebox'})
    team_abbr = score_table.find_all('strong')
    game['away_abbr'] = team_abbr[0].find('a')['href'].split('/')[2]
    game['home_abbr'] = team_abbr[1].find('a')['href'].split('/')[2]
    date_time = soup.find('div', {'class':'scorebox_meta'}).findAll('div')
    game['date'] = date_time[0].text
    game['time'] = date_time[1].text
    return game

def batting_stats(soup,table_number):
    s = soup.findAll('div', {'class':'table_wrapper'})
    for comment in s[table_number].find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            table = BS(comment)
    for r in table.find('table').findAll('tr'):
        summary = {x['data-stat']:x.text.strip() for x in r.findAll('td')}
    
    return summary

def pitching_stats(soup, table_number):
    s = soup.findAll('div', {'class':"section_wrapper setup_commented commented"})[1]

    for comment in s.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            table = BS(comment)
    for r in table.findAll('table')[table_number].findAll('tr'):
        summary = {x['data-stat']:x.text.strip() for x in r.findAll('td')}

    return summary

def pitchers(soup, table_number):
    s = soup.findAll('div', {'class':"section_wrapper setup_commented commented"})[1]
    data = []
    for comment in s.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            table = BS(comment)
    for r in table.findAll('table')[table_number].findAll('tr')[1:-1]:
        summary = {x['data-stat']:x.text.strip() for x in r.findAll('td')}
        summary['name'] = r.find('th',{'data-stat':'player'}).find('a')['href'].split('/')[-1][:-6].strip()
        data.append(summary)
    return data

def get_all_data(url):
    res = requests.get(url)
    game_id = url.split('/')[-1][:-6]
    soup = BS(res.text,'html.parser')
    
    data = {
        'game': get_game(soup, game_id),
        'away_batting': batting_stats(soup, 0),
        'home_batting':batting_stats(soup, 1),
        'away_pitching':pitching_stats(soup, 0),
        'home_pitching':pitching_stats(soup, 1),
        'away_pitchers': pitchers(soup, 0),
        'home_pitchers': pitchers(soup, 1)
    }
    
    return data

In [15]:
url = 'https://www.baseball-reference.com/leagues/majors/2022-schedule.shtml'

In [21]:
r = requests.get(url)

In [22]:
soup = BS(r.text)
html = soup.find_all('a', text = 'Boxscore')
boxscores2022 = [x['href'] for x in html]

In [23]:
boxscores2022

['/boxes/ANA/ANA202204070.shtml',
 '/boxes/ARI/ARI202204070.shtml',
 '/boxes/ATL/ATL202204070.shtml',
 '/boxes/CHN/CHN202204070.shtml',
 '/boxes/KCA/KCA202204070.shtml',
 '/boxes/SLN/SLN202204070.shtml',
 '/boxes/WAS/WAS202204070.shtml',
 '/boxes/ANA/ANA202204080.shtml',
 '/boxes/ARI/ARI202204080.shtml',
 '/boxes/ATL/ATL202204080.shtml',
 '/boxes/COL/COL202204080.shtml',
 '/boxes/DET/DET202204080.shtml',
 '/boxes/MIN/MIN202204080.shtml',
 '/boxes/NYA/NYA202204080.shtml',
 '/boxes/PHI/PHI202204080.shtml',
 '/boxes/SFN/SFN202204080.shtml',
 '/boxes/TBA/TBA202204080.shtml',
 '/boxes/TOR/TOR202204080.shtml',
 '/boxes/WAS/WAS202204080.shtml',
 '/boxes/ANA/ANA202204090.shtml',
 '/boxes/ARI/ARI202204090.shtml',
 '/boxes/ATL/ATL202204090.shtml',
 '/boxes/CHN/CHN202204090.shtml',
 '/boxes/COL/COL202204090.shtml',
 '/boxes/DET/DET202204090.shtml',
 '/boxes/KCA/KCA202204090.shtml',
 '/boxes/MIN/MIN202204090.shtml',
 '/boxes/NYA/NYA202204090.shtml',
 '/boxes/PHI/PHI202204090.shtml',
 '/boxes/SFN/S

In [28]:
bad_links = []
games2022 = []
for link in tqdm(boxscores2022, desc = 'tqdm() Progress Bar'):
    try:
        time.sleep(0.5)
        url = 'https://www.baseball-reference.com' + link
        games2022.append(get_all_data(url))
    except UnboundLocalError:
        bad_links.append(link)

tqdm() Progress Bar: 100%|███████████████████████████████████████████████████████████| 845/845 [10:59<00:00,  1.28it/s]


In [66]:
def get_todays_games(url):
    today_games=[]
    games_today = games2022[:845]
    page = requests.get(url).text
    soup = BS(page)
    summaries = soup.findAll('div', {'class':'game_summary'})
    for s in summaries:
        game = {
            'game':{
                'game_id': s.find('a', text='Preview')['href'][-18:-6],
                'is_test':True,
                'date': dt.datetime.now().strftime('%A, %B %d, %Y')
            },
            'home_batting':{'R':0},
            'away_batting':{'R':0},
            'home_pitching':{'R':0},
            'away_pitching':{'R':0},
            'home_pitchers':[{'R':0}],
            'away_pitchers':[{'R':0}]
        }
        cells = s.findAll('table')[0].findAll('td')

        # skip postponed games
        if not s.find('a', text="Preview"): continue

        # skip game 2 in double header - links look like this for 2nd games: "/previews/2020/PHI202008051.shtml"
        if s.find('a', text="Preview")['href'][-7]=='2': continue

        try:
            team_links = s.findAll('a')
            game['game']['away_abbr'] = team_links[0]['href'].split('/')[2]
            game['game']['home_abbr'] = team_links[2]['href'].split('/')[2]
        except Exception as e:
            #just all star games trigger this, I think
            print(team_links)
            continue



        # get pitchers
        try:
            cells = s.findAll('table')[1].findAll('td')
            game['away_pitchers'][0]['name'] = cells[1].find('a')['href'].split('/')[-1][:-6].strip()
            game['home_pitchers'][0]['name'] = cells[3].find('a')['href'].split('/')[-1][:-6].strip()
        except Exception as e:
            # no pitcher
            game['away_pitchers'][0]['name'] = ''
            game['home_pitchers'][0]['name'] = ''
        today_games.append(game)

    games_today.extend(today_games)
    print(len(today_games), "Games today")
    for x in today_games: print(x['game']['game_id'],
                                x['game']['away_abbr'],x['away_pitchers'][0]['name'],
                                x['game']['home_abbr'],x['home_pitchers'][0]['name'])
    return games_today

In [67]:
def rolling_stats(period, df, stat_col):
    for s in stat_col:
        #if 'object' in str(df[s].dtype): continue
        df[s+'_'+str(period)+'_Avg'] = df.groupby('team')[s].apply(lambda x: x.rolling(period).mean())
        df[s+'_'+str(period)+'_std'] = df.groupby('team')[s].apply(lambda x: x.rolling(period).std())
        
    return df


def get_diff_df(df, name, is_pitcher = False):
    df['date'] = pd.to_datetime(df['game_id'].str[3:-1], format='%Y%m%d')
    df = df.sort_values('date')
    df = df.set_index(df.groupby('date')['date'].apply(lambda x: x + np.arange(x.size).astype(np.timedelta64)))
    stat_columns = [x for x in df.columns if 'float' in str(df[x].dtype)]
    
    df = rolling_stats('5d', df, stat_columns)
    df = rolling_stats('10d', df, stat_columns)
    df = rolling_stats('45d', df, stat_columns)
    df = rolling_stats('180d', df, stat_columns)
    
    
    df.drop(columns = stat_columns, inplace = True)
    stat_columns = [x for x in df.columns if 'float' in str(df[x].dtype)]
    df = df.reset_index(drop = True)
    df = df.sort_values('date')
    
    
    for s in stat_columns:
        if is_pitcher:
            df[s] = df.groupby('name')[s].shift(1)
        else:
            df[s] = df.groupby('team')[s].shift(1)
            
            
    away_df = df[df['home?'] == False].copy()
    away_df = away_df.set_index('game_id')
    away_df = away_df[stat_columns]
    
    
    
    home_df = df[df['home?'] == True].copy()
    home_df = home_df.set_index('game_id')
    home_df = home_df[stat_columns]
    
    diff = home_df.subtract(away_df, fill_value = 0)
    diff = diff.reset_index()
    
    for s in stat_columns:
        colname = str(name + '_' + s)
        diff[colname] = diff[s]
        diff.drop(columns = s, inplace = True)
        
    return diff

In [68]:
numeric_stats = ['AB', 'R', 'H', 'RBI',
                 'BB','SO', 'PA', 'batting_avg',
                 'onbase_perc','slugging_perc','onbase_plus_slugging',
                 'pitches', 'strikes_total', 'wpa_bat','leverage_index_avg']

numeric_pitchers = ['IP',
 'H',
 'R',
 'ER',
 'BB',
 'SO',
 'HR',
 'earned_run_avg',
 'batters_faced',
 'pitches',
 'strikes_total',
 'strikes_contact',
 'strikes_swinging',
 'strikes_looking',
 'inplay_gb_total',
 'inplay_fb_total',
 'inplay_ld',
 'inplay_unk',
 'game_score',
 'inherited_runners',
 'inherited_score',
 'wpa_def',
 'leverage_index_avg',
 'cwpa_def',
 'cli_avg',
 're24_def']

numeric_pitching = ['IP',
 'H',
 'R',
 'ER',
 'BB',
 'SO',
 'HR',
 'earned_run_avg',
 'batters_faced',
 'pitches',
 'strikes_total',
 'strikes_contact',
 'strikes_swinging',
 'strikes_looking',
 'inplay_gb_total',
 'inplay_fb_total',
 'inplay_ld',
 'inplay_unk',
 'game_score',
 'inherited_runners',
 'inherited_score',
 'wpa_def',
 'leverage_index_avg',
 'cwpa_def',
 'cli_avg',
 're24_def']


def get_todays_game_df(dictionary):
    games_today = []

    for g in dictionary:
        game_summary = g['game']
        if 'is_test' not in game_summary.keys(): game_summary['is_test'] = False
        game_summary['home_pitcher'] = g['home_pitchers'][0]['name']
        game_summary['away_pitcher'] = g['away_pitchers'][0]['name']
        game_summary['home_score'] = g['home_batting']['R']
        game_summary['away_score'] = g['away_batting']['R']
        games_today.append(game_summary)
    
    return pd.DataFrame(games_today)

def get_todays_stats(dictionary):
    batting_today = []
    pitching_today = []
    pitchers_today = []
    for g in dictionary:
        pairs = [
            ('away_batting', batting_today),
            ('home_batting', batting_today),
            ('away_pitching', pitching_today),
            ('home_pitching', pitching_today),
            ('away_pitchers',pitchers_today ),
            ('home_pitchers', pitchers_today)
        ]
        for key, d in pairs:
            if isinstance(g[key], list):
                for x in g[key]:
                    if 'home' in key:
                        x['home?'] = True
                        x['team']=g['game']['home_abbr']
                    else:
                        x['home?'] = False
                        x['team'] = g['game']['away_abbr']
                    x['game_id'] = g['game']['game_id']
                    d.append(x)

            else:
                x = g[key]
                if 'home' in key:
                    x['home?'] = True
                    x['team'] = g['game']['home_abbr']
                else:
                    x['home?'] = False
                    x['team'] = g['game']['away_abbr']
                x['game_id'] = g['game']['game_id']
                d.append(x)
    return pd.DataFrame(batting_today), pd.DataFrame(pitching_today), pd.DataFrame(pitchers_today)

def get_todays_rolling(dictionary):
    df = pd.DataFrame(get_todays_game_df(dictionary))
    batting_today, pitching_today, pitchers_today = get_todays_stats(dictionary)
    df[['day', 'del1','del2','date']] = df['date'].str.extract('(\w+)(,)(\s+)(\w+\s+\d+\W+\s+\d+)', expand=True )
    df = df.drop(columns = ['del1','del2'])
    df['date'] = pd.to_datetime(df['date'])
    cond = (df['home_score']>df['away_score'], df['home_score']<df['away_score'], df['home_score'] == df['away_score'])
    choices = (True, False, np.nan)
    df['Home_win'] = np.select(cond, choices)
    for x in numeric_pitching:
        try:
            pitching_today[x] = pitching_today[x].astype(float)
        except ValueError:
            pass

    for x in numeric_pitchers:
        try:
            pitchers_today[x] = pitchers_today[x].astype(float)
        except ValueError:
            pass

    for stat in numeric_stats:
        batting_today[stat] = batting_today[stat].astype(float)
        
    starters_today = pitchers_today.groupby(['team', 'game_id']).head(1)
    starters_today = pd.merge(starters_today, df[['game_id','Home_win']]).drop_duplicates()
    starters_today = starters_today[starters_today['earned_run_avg'] != '']
    starters_today['earned_run_avg'] = starters_today['earned_run_avg'].astype(float)
    
    df = pd.merge(left = df, right = get_diff_df(batting_today, 'batting'), on ='game_id', how = 'left')
    df = pd.merge(left = df, right = get_diff_df(pitching_today, 'pitching'), on ='game_id', how = 'left')
    df = pd.merge(left = df, right = get_diff_df(starters_today, 'pitchers', is_pitcher = True), on ='game_id', how = 'left')
    
    starters_today['rest'] = starters_today.groupby('name')['date'].diff().dt.days
    home_starters_today = starters_today[starters_today['home?'] == True]
    away_starters_today = starters_today[starters_today['home?']==False]

    home_starters_today = home_starters_today.rename(columns={'rest':'home_rest'})
    away_starters_today = away_starters_today.rename(columns={'rest':'away_rest'})
    
    df = pd.merge(df, 
        home_starters_today[['game_id', 'name', 'home_rest']], 
         left_on = ['game_id', 'home_pitcher'],
        right_on = ['game_id', 'name'],
        how = 'left')
    df = pd.merge(df, 
        away_starters_today[['game_id', 'name', 'away_rest']], 
         left_on = ['game_id', 'home_pitcher'],
        right_on = ['game_id', 'name'],
        how = 'left')
    df['rest_diff_home-away'] = df['home_rest'] - df['away_rest']
    df = df.drop(columns=['away_rest', 'home_rest'])
    df = df.drop(columns=['name_x','name_y'])
    
    return df

In [69]:
def add_win_pct(df):
    df['away_losses'] = pd.DataFrame(df.groupby([pd.Grouper(key = 'date', freq = '1Y'), 'away_abbr'])['Home_win'].cumsum())
    df['home_wins'] = pd.DataFrame(df.groupby([pd.Grouper(key = 'date', freq = '1Y'), 'home_abbr'])['Home_win'].cumsum())
    df['num_games'] = 1
    df['total_home_games'] = pd.DataFrame(df.groupby([pd.Grouper(key = 'date', freq = '1Y'), 'home_abbr'])['num_games'].cumsum())
    df['total_away_games'] = pd.DataFrame(df.groupby([pd.Grouper(key = 'date', freq = '1Y'), 'away_abbr'])['num_games'].cumsum())
    df['away_win_pct'] = (1 - (df['away_losses']/ df['total_away_games']))
    df['home_win_pct'] = (df['home_wins']/df['total_home_games'])
    df['win_pct_diff'] = df['home_win_pct'] - df['away_win_pct']
    
    return df
    
def get_todays_predictions(df):
    df = add_win_pct(df)
    df2 = df[df['Home_win'].isna()]
    df2['prediction'] = full_hist_class.predict(df2[numerics])
    df2[['prob_away_win', 'prob_home_win']] = full_hist_class.predict_proba(df2[numerics])
    
    return df2[['away_abbr', 'home_abbr', 'prediction', 'prob_away_win', 'prob_home_win']].sort_values('prob_home_win', ascending = False)

In [70]:
url_for_todays_games = 'https://www.baseball-reference.com/previews/'

# June 9

In [72]:
get_todays_predictions(get_todays_rolling(get_todays_games(url_for_todays_games)))

11 Games today
CIN202206090 ARI davieza02 CIN mahlety01
TBA202206090 STL mikolmi01 TBR mcclash01
CHA202206090 LAD anderty01 CHW ceasedy01
MIL202206090 PHI eflinza01 MIL burneco01
SFN202206090 COL gombeau01 SFG webblo01
MIA202206090 WSN strasst01 MIA rogertr01
CLE202206090 OAK kaprija01 CLE pilkiko01
ATL202206090 PIT brubajt01 ATL friedma01
MIN202206090 NYY colege01 MIN bundydy01
KCA202206090 BAL lylesjo01 KCR bubickr01
ANA202206090 BOS pivetni01 LAA ohtansh01


<ipython-input-69-bcfb3599f1d7>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['prediction'] = full_hist_class.predict(df2[numerics])
<ipython-input-69-bcfb3599f1d7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['prob_away_win', 'prob_home_win']] = full_hist_class.predict_proba(df2[numerics])
C:\Users\jacob\anaconda3\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

,away_abbr,home_abbr,prediction,prob_away_win,prob_home_win
845,ARI,CIN,1,0.316238,0.683762
846,STL,TBR,1,0.360804,0.639196
850,WSN,MIA,1,0.367317,0.632683
855,BOS,LAA,1,0.405151,0.594849
849,COL,SFG,1,0.408069,0.591931
853,NYY,MIN,1,0.409285,0.590715
854,BAL,KCR,1,0.410140,0.589860
852,PIT,ATL,1,0.416363,0.583637
848,PHI,MIL,1,0.470167,0.529833
847,LAD,CHW,1,0.498219,0.501781
